In [1]:
import pandas as pd
import glob
import re
import numpy as np

The GHCN files are in a fixed-width format. Each row represents a month of data for a variable recorded at a station. The first four columns contain the station id, year, month, and element being measured. The remaining columns are the daily value (VALUE1) and 3 daily flags (MFLAG1, QFLAG1, SFLAG1). Each day has these four columns with the number being the day of the month.

The tedious part of this is generating the list of column widths so the data will be separated correctly, and generating the list of column names. The pattern for both is specified in the data readme.

Code to generate widths argument:

In [2]:
# initial identifier columns
widths = [11, 4, 2, 4]

# generate column widths for 31 days
for i in range(1, 32):
    exend_widths = [5, 1, 1, 1]
    widths.extend(exend_widths)

Code to generate names argument:

In [3]:
# initial identifier columns
names = ['id', 'year', 'month', 'element']

# generate names for daily columns
for i in range(1, 32):
    extend_names = [f'value{i}', f'mflag{i}', f'qflag{i}', f'sflag{i}']
    names.extend(extend_names)

In [4]:
test = pd.read_fwf('../data/USC00401553_cedar_hill.dly', widths = widths, header = None, names = names)

In [5]:
test.head()

,id,year,month,element,value1,mflag1,qflag1,sflag1,value2,mflag2,...,qflag29,sflag29,value30,mflag30,qflag30,sflag30,value31,mflag31,qflag31,sflag31
0,USC00401553,1897,3,TMAX,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
1,USC00401553,1897,3,TMIN,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
2,USC00401553,1897,3,PRCP,0,P,NaN,6.0,0,P,...,NaN,6.0,198,NaN,NaN,6.0,203,NaN,NaN,6.0
3,USC00401553,1897,3,SNOW,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
4,USC00401553,1897,4,TMAX,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN


Process all files in folder into a dataframe:

In [6]:
# list of the files
files = glob.glob('../data/*.dly')

# empty list to store dataframes
df_list = []

# loop over the files to create dataframes
for file in files:
    df_list.append(pd.read_fwf(file, widths = widths, header = None, names = names))

# concatenate to single dataframe
weather = pd.concat(df_list, ignore_index = True)

In [7]:
weather.tail()

,id,year,month,element,value1,mflag1,qflag1,sflag1,value2,mflag2,...,qflag29,sflag29,value30,mflag30,qflag30,sflag30,value31,mflag31,qflag31,sflag31
79280,USC00405349,1954,5,SNOW,0,NaN,NaN,0.0,0,NaN,...,NaN,0.0,0,NaN,NaN,0.0,0,NaN,NaN,0.0
79281,USC00405349,1954,5,SNWD,0,NaN,NaN,0.0,0,NaN,...,NaN,0.0,0,NaN,NaN,0.0,0,NaN,NaN,0.0
79282,USC00405349,1954,6,PRCP,28,NaN,NaN,0.0,0,NaN,...,NaN,0.0,0,NaN,NaN,0.0,-9999,NaN,NaN,NaN
79283,USC00405349,1954,6,SNOW,0,NaN,NaN,0.0,0,NaN,...,NaN,0.0,0,NaN,NaN,0.0,-9999,NaN,NaN,NaN
79284,USC00405349,1954,6,SNWD,0,NaN,NaN,0.0,0,NaN,...,NaN,0.0,0,NaN,NaN,0.0,-9999,NaN,NaN,NaN


Melt and pivot dataframe so that there is a row per station per day:

In [8]:
# columns for melting
id_vars = ['id', 'year', 'month', 'element']
value_vars = weather.columns[4:-1]
var_name = 'day'
value_name = 'day_value'

weather_melt = pd.melt(weather, 
                       id_vars = id_vars,
                       value_vars = value_vars,
                       var_name = var_name,
                       value_name = value_name)

In [9]:
weather_melt.head()

,id,year,month,element,day,day_value
0,USC00408414,1941,2,PRCP,value1,-9999
1,USC00408414,1941,2,SNOW,value1,-9999
2,USC00408414,1941,2,SNWD,value1,-9999
3,USC00408414,1941,2,WT18,value1,-9999
4,USC00408414,1941,3,PRCP,value1,0


Drop 'flag' rows:

In [10]:
weather_melt = weather_melt[~weather_melt['day'].str.contains('flag')]

Create date column from year, month, and extracted number from day column:

In [11]:
# day_num column from day
weather_melt['day_num'] = weather_melt['day'].str.extract('(\d+)')[0].str.zfill(2)

In [12]:
# same for month_num
weather_melt['month_num'] = weather_melt['month'].astype(str).str.zfill(2)

In [13]:
# concatenate into date column
weather_melt['date'] =  (weather_melt['year'].astype(str) + '-' + 
                         weather_melt['month_num'].astype(str) + '-' + 
                         weather_melt['day_num'].astype(str))

In [14]:
weather_melt.head()

,id,year,month,element,day,day_value,day_num,month_num,date
0,USC00408414,1941,2,PRCP,value1,-9999,01,02,1941-02-01
1,USC00408414,1941,2,SNOW,value1,-9999,01,02,1941-02-01
2,USC00408414,1941,2,SNWD,value1,-9999,01,02,1941-02-01
3,USC00408414,1941,2,WT18,value1,-9999,01,02,1941-02-01
4,USC00408414,1941,3,PRCP,value1,0,01,03,1941-03-01


Drop unneeded columns:

In [15]:
weather_melt = weather_melt.drop(['day', 'day_num', 'month_num'], axis = 1)

In [16]:
weather_melt.head()

,id,year,month,element,day_value,date
0,USC00408414,1941,2,PRCP,-9999,1941-02-01
1,USC00408414,1941,2,SNOW,-9999,1941-02-01
2,USC00408414,1941,2,SNWD,-9999,1941-02-01
3,USC00408414,1941,2,WT18,-9999,1941-02-01
4,USC00408414,1941,3,PRCP,0,1941-03-01


Replace missing observation value (-9999) with null. (Was previously dropping entirely but doing it this way will maintain a record for each day for each station instead of having days missing entirely.)

In [17]:
weather_melt['day_value'] = weather_melt['day_value'].replace(-9999, np.NaN)

In [18]:
weather_melt.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2457835 entries, 0 to 9593484
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   id         2457835 non-null  object 
 1   year       2457835 non-null  int64  
 2   month      2457835 non-null  int64  
 3   element    2457835 non-null  object 
 4   day_value  2110276 non-null  float64
 5   date       2457835 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 131.3+ MB


In [19]:
# core element measurements
element_keep = ['PRCP', 'TMIN', 'TMAX', 'SNOW', 'SNWD']

Keep only rows with elements of interest:

In [20]:
weather_melt = weather_melt[weather_melt['element'].isin(element_keep)]

In [21]:
weather_melt.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1522069 entries, 0 to 9593484
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   id         1522069 non-null  object 
 1   year       1522069 non-null  int64  
 2   month      1522069 non-null  int64  
 3   element    1522069 non-null  object 
 4   day_value  1444359 non-null  float64
 5   date       1522069 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 81.3+ MB


In [22]:
weather_pivot =  weather_melt.pivot_table('day_value', index = ['id', 'year', 'month', 'date'], columns = 'element').reset_index()

In [23]:
weather_pivot.head()

element,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN
0,USC00401553,1897,3,1897-03-01,0.0,NaN,NaN,NaN,NaN
1,USC00401553,1897,3,1897-03-02,0.0,NaN,NaN,NaN,NaN
2,USC00401553,1897,3,1897-03-03,117.0,NaN,NaN,NaN,NaN
3,USC00401553,1897,3,1897-03-04,0.0,NaN,NaN,NaN,NaN
4,USC00401553,1897,3,1897-03-05,109.0,NaN,NaN,NaN,NaN


In [24]:
weather_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360284 entries, 0 to 360283
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      360284 non-null  object 
 1   year    360284 non-null  int64  
 2   month   360284 non-null  int64  
 3   date    360284 non-null  object 
 4   PRCP    357112 non-null  float64
 5   SNOW    277259 non-null  float64
 6   SNWD    248420 non-null  float64
 7   TMAX    280780 non-null  float64
 8   TMIN    280788 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 24.7+ MB


In [25]:
weather_pivot.groupby(['id', 'year']).count().reset_index().sort_values('year').head(30)

element,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN
467,USC00406371,1890,31,31,31,0,0,1,1
468,USC00406371,1891,101,101,101,0,0,29,0
104,USC00403280,1891,153,153,153,2,0,0,0
105,USC00403280,1893,365,365,362,93,0,346,346
469,USC00406371,1893,342,342,319,121,0,87,87
470,USC00406371,1894,279,279,233,35,0,183,177
106,USC00403280,1894,304,304,304,122,0,295,295
471,USC00406371,1895,365,365,324,123,0,360,362
107,USC00403280,1895,303,303,302,151,0,303,303
472,USC00406371,1896,359,359,315,91,0,354,352


Thoughts on handling stations with spotty data:

Part of the benefit of using stations in relatively close proximity is to help fill in for data that might be missing at one station or another. I also only chose stations that had a reasonably long history and also high percent data completion. The real question might be to decide when to make the early cut off year-wise. Two stations have data starting in the 1890s but the records for both are spotty from the early years. It looks like 1898 is when those two stations become more reliable, and a third starts gathering data in that year and becomes complete in 1899 so 1899 or 1900 seems like a good starting point.

For handling mostly complete stations with a measurement missing here and there: I could take an average of the day before and the day after (at least with temperature) or just not consider nulls at all since I'll have data from several stations.

I'll need to do more detailed analysis to see if the snow data is complete enough to be useful. It is one of the core observations but there will necessarily be less data than temperature or general precipitation. My assumption is that this measurement will be more accurate for later time periods while precipitation and temperature should be fairly reliable even early in the time period covered by this data.

In [26]:
weather_pivot = weather_pivot[(weather_pivot['year'] >= 1899) & (weather_pivot['year'] < 2024)]

Converting measurements to standard:
- PRCP from tenths of mm to cm (inches might be better since it's standard in the US)
- SNOW and SNWD from mm to cm (or inches)
- TMAX and TMIN from tenths of degrees C to C (or F)

I'm leaning toward converting to US units just because it will make for better communication of data with the expected audience.

In [27]:
# PRCP in inches

weather_pivot['prcp_inches'] = weather_pivot['PRCP'].apply(lambda x : x / 254)

Interesting data quality issue:

There are ~40 rows where the max and min temperatures look like theye were reversed in the original data (out of ~360K rows). When converting the temps to F I'm going to address this by taking whichever is the higher value from the two columns as the max and the lower value as the min. There are approximately the same number of rows where the values are equal, so the conditional function will need to take that into account.

In [28]:
# function to convert tenths of degrees Celcius to Fahrenheit
def convert_to_f(row):
    if row['TMAX'] >= row['TMIN']:
        return ((row['TMAX'] / 10) * (9 / 5)) + 32, ((row['TMIN'] / 10) * (9 / 5)) + 32
    else:
        return ((row['TMIN'] / 10) * (9 / 5)) + 32, ((row['TMAX'] / 10) * (9 / 5)) + 32

# apply function across the dataframe
weather_pivot[['tmax_f', 'tmin_f']] = weather_pivot.apply(convert_to_f, 
                                                          axis=1, 
                                                          result_type='expand')

In [29]:
weather_pivot.head()

element,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN,prcp_inches,tmax_f,tmin_f
459,USC00401553,1899,1,1899-01-01,0.0,0.0,25.0,-33.0,-172.0,0.000000,26.06,1.04
460,USC00401553,1899,1,1899-01-02,0.0,0.0,25.0,44.0,-150.0,0.000000,39.92,5.00
461,USC00401553,1899,1,1899-01-03,0.0,0.0,NaN,78.0,17.0,0.000000,46.04,35.06
462,USC00401553,1899,1,1899-01-04,508.0,0.0,NaN,178.0,78.0,2.000000,64.04,46.04
463,USC00401553,1899,1,1899-01-05,64.0,0.0,NaN,133.0,0.0,0.251969,55.94,32.00


With the data mostly clean, I want to bring in station metadata:

In [38]:
station_metadata = pd.read_fwf('../data/ghcnd-stations.txt', 
                               names = ['ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
                                        'GSN FLAG', 'HCN/CRN FLAG'])

In [39]:

station_metadata.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,NaN,GSN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,NaN,NaN,41217.0


In [37]:
# list of stations of interest
station_list = weather_pivot['id'].unique().tolist()

# filter to just those stations
station_metadata = station_metadata[station_metadata['ID'].isin(station_list)]

In [43]:
# drop unneeded columns from station data before merge

station_metadata = station_metadata.drop(['ELEVATION', 'NAME', 'GSN FLAG', 'HCN/CRN FLAG'], axis = 1).\
    rename({'ID': 'id', 'LATITUDE': 'lat', 'LONGITUDE': 'long', 'STATE': 'station_name'}, axis = 1)